# 🔬 TPN-RAG Full Evaluation Notebook

## Advantages of Notebook Evaluation:
- ✅ **Load models ONCE** - keep in GPU memory
- ✅ **Run incrementally** - test 10 samples, then 100, then all
- ✅ **Checkpoint progress** - save every N samples
- ✅ **Interactive analysis** - explore results in real-time
- ✅ **Resume on failure** - don't lose progress

---

## 1️⃣ Setup & Configuration

In [ ]:
# Configuration - EDIT THESE
NUM_SAMPLES = 100  # Start with 100, increase to 941 for full eval
TOP_K = 10  # Retrieval depth (5 was too low)
CHECKPOINT_EVERY = 10  # Save results every N samples
OUTPUT_FILE = "full_eval_results.json"

# Model configuration
LLM_MODEL = "chandramax/tpn-gpt-oss-20b"
EMBED_MODEL = "Qwen/Qwen3-Embedding-8B"
JUDGE_MODEL = "gpt-5-mini"  # For DeepEval metrics

print(f"📊 Will evaluate {NUM_SAMPLES} samples with top_k={TOP_K}")

In [ ]:
# Imports
import json
import os
import sys
import time
from pathlib import Path
from typing import Dict, List, Tuple, Optional
from dataclasses import dataclass, asdict
import re
from tqdm.notebook import tqdm

# Set environment
os.environ['TOKENIZERS_PARALLELISM'] = 'false'

# Add project root
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

print(f"✅ Project root: {project_root}")

## 2️⃣ Load Models ONCE (Keep in Memory)

In [ ]:
# Load LLM Model (20B) - ONLY RUN ONCE!
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

print(f"🔄 Loading LLM: {LLM_MODEL}...")
start = time.time()

tokenizer = AutoTokenizer.from_pretrained(LLM_MODEL, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    LLM_MODEL,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True
)

print(f"✅ LLM loaded in {time.time()-start:.1f}s")
print(f"   Device: {next(model.parameters()).device}")
print(f"   Memory: {torch.cuda.memory_allocated()/1e9:.1f} GB")

In [ ]:
# Load Embedding Model - ONLY RUN ONCE!
from sentence_transformers import SentenceTransformer

print(f"🔄 Loading embedding model: {EMBED_MODEL}...")
start = time.time()

embed_model = SentenceTransformer(
    EMBED_MODEL,
    trust_remote_code=True,
    model_kwargs={"torch_dtype": torch.bfloat16}
)

print(f"✅ Embedding model loaded in {time.time()-start:.1f}s")

In [ ]:
# Load ChromaDB - ONLY RUN ONCE!
import chromadb

chroma_path = project_root / "data" / "chroma"
client = chromadb.PersistentClient(path=str(chroma_path))
collection = client.get_collection("tpn_documents")

print(f"✅ ChromaDB loaded: {collection.count()} documents")

## 3️⃣ Load Test Dataset

In [ ]:
# Load test data
test_file = project_root / "eval" / "data" / "test_with_citations.jsonl"

samples = []
with open(test_file, 'r') as f:
    for line in f:
        samples.append(json.loads(line))

print(f"✅ Loaded {len(samples)} test samples")
print(f"📊 Will evaluate first {min(NUM_SAMPLES, len(samples))} samples")

In [ ]:
# Preview first sample
sample = samples[0]
for msg in sample['messages']:
    print(f"[{msg['role']}]: {msg['content'][:200]}...")
    print()

## 4️⃣ Helper Functions

In [ ]:
def extract_qa(sample: dict) -> Tuple[str, str]:
    """Extract question and expected answer from sample."""
    question = None
    expected = None
    for msg in sample['messages']:
        if msg['role'] == 'user':
            question = msg['content']
        elif msg['role'] == 'assistant':
            expected = msg.get('content', '')
    return question, expected


def retrieve_context(question: str, top_k: int = TOP_K) -> Tuple[str, List[str], List[Dict]]:
    """Retrieve relevant context from ChromaDB."""
    query_embedding = embed_model.encode([question], prompt_name="query")[0].tolist()
    
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=top_k,
        include=["documents", "metadatas", "distances"]
    )
    
    context_parts = []
    context_list = []
    sources = []
    
    for i in range(len(results['documents'][0])):
        doc = results['documents'][0][i]
        meta = results['metadatas'][0][i] if results['metadatas'] else {}
        distance = results['distances'][0][i] if results['distances'] else 0
        
        source_name = meta.get('source', meta.get('document_name', 'Unknown'))
        score = 1 - distance
        
        context_parts.append(f"[Source: {source_name}]\n{doc}")
        context_list.append(doc)
        sources.append({'source': source_name, 'score': score})
    
    return "\n\n---\n\n".join(context_parts), context_list, sources


def run_inference(question: str, context: Optional[str] = None, max_tokens: int = 1024) -> str:
    """Run model inference with or without context."""
    if context:
        system_prompt = f"""You are a clinical expert in neonatal/pediatric TPN.
Use these reference documents to answer accurately with citations:

<reference_documents>
{context}
</reference_documents>

Always cite sources as [Source Name: "quote"]."""
    else:
        system_prompt = "You are a clinical expert in neonatal/pediatric TPN. Answer accurately."
    
    messages = [
        {"role": "developer", "content": system_prompt},
        {"role": "user", "content": question}
    ]
    
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_tokens,
            temperature=0.1,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )
    
    response = tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)
    return response.strip()

print("✅ Helper functions defined")

## 5️⃣ Metric Functions

In [ ]:
# Deterministic metrics (fast, no API calls)
from nltk.tokenize import word_tokenize
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer
import nltk

# Download NLTK data
nltk.download('punkt', quiet=True)
nltk.download('punkt_tab', quiet=True)

rouge = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
smoother = SmoothingFunction().method1


def compute_deterministic(reference: str, hypothesis: str) -> Dict:
    """Compute BLEU, ROUGE, F1 (fast, no API)."""
    ref_tokens = word_tokenize(reference.lower())
    hyp_tokens = word_tokenize(hypothesis.lower())
    
    if len(hyp_tokens) == 0:
        return {'bleu_1': 0, 'bleu_4': 0, 'rouge_l': 0, 'f1': 0}
    
    bleu_1 = sentence_bleu([ref_tokens], hyp_tokens, weights=(1,0,0,0), smoothing_function=smoother)
    bleu_4 = sentence_bleu([ref_tokens], hyp_tokens, weights=(0.25,0.25,0.25,0.25), smoothing_function=smoother)
    rouge_l = rouge.score(reference, hypothesis)['rougeL'].fmeasure
    
    # F1
    ref_set = set(ref_tokens)
    hyp_set = set(hyp_tokens)
    common = ref_set & hyp_set
    precision = len(common) / len(hyp_set) if hyp_set else 0
    recall = len(common) / len(ref_set) if ref_set else 0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
    
    return {'bleu_1': bleu_1, 'bleu_4': bleu_4, 'rouge_l': rouge_l, 'f1': f1}

print("✅ Deterministic metrics ready")

In [ ]:
# LLM-based metrics (DeepEval - uses API)
from deepeval.metrics import GEval, FaithfulnessMetric, AnswerRelevancyMetric, ContextualRecallMetric
from deepeval.test_case import LLMTestCase, LLMTestCaseParams

# Clinical correctness GEval
clinical_geval = GEval(
    name="Clinical Correctness",
    criteria="""Evaluate clinical correctness for TPN with emphasis on:
1. DOSING VALUES: Are numeric values exactly correct?
2. UNITS: Are units correct? (mg vs g, mEq vs mmol)
3. RANGES: Are ranges accurate?
4. POPULATIONS: Is patient population correct?
5. SAFETY: Are contraindications mentioned?""",
    evaluation_params=[LLMTestCaseParams.ACTUAL_OUTPUT, LLMTestCaseParams.EXPECTED_OUTPUT],
    model=JUDGE_MODEL,
    threshold=0.7
)

faithfulness_metric = FaithfulnessMetric(threshold=0.7, model=JUDGE_MODEL)
relevancy_metric = AnswerRelevancyMetric(threshold=0.7, model=JUDGE_MODEL)
recall_metric = ContextualRecallMetric(threshold=0.7, model=JUDGE_MODEL)

print("✅ DeepEval metrics ready")

## 6️⃣ Run Evaluation (with Checkpointing)

In [ ]:
# Initialize results
results = []
start_idx = 0

# Check for existing checkpoint
checkpoint_file = Path(OUTPUT_FILE.replace('.json', '_checkpoint.json'))
if checkpoint_file.exists():
    with open(checkpoint_file, 'r') as f:
        checkpoint = json.load(f)
        results = checkpoint['results']
        start_idx = len(results)
        print(f"📂 Resuming from checkpoint: {start_idx} samples already done")
else:
    print(f"🆕 Starting fresh evaluation")

print(f"📊 Will evaluate samples {start_idx} to {min(NUM_SAMPLES, len(samples))-1}")

In [ ]:
# MAIN EVALUATION LOOP
num_to_eval = min(NUM_SAMPLES, len(samples))

for idx in tqdm(range(start_idx, num_to_eval), desc="Evaluating"):
    sample = samples[idx]
    question, expected = extract_qa(sample)
    
    try:
        # Phase 1: Model only (no RAG)
        phase1_answer = run_inference(question, context=None)
        
        # Retrieve context
        context, context_list, sources = retrieve_context(question, top_k=TOP_K)
        
        # Phase 2: Model + RAG
        phase2_answer = run_inference(question, context=context)
        
        # Deterministic metrics (fast)
        p1_det = compute_deterministic(expected, phase1_answer)
        p2_det = compute_deterministic(expected, phase2_answer)
        
        # LLM metrics (slower, API calls)
        # Phase 1 clinical
        tc1 = LLMTestCase(input=question, actual_output=phase1_answer, expected_output=expected)
        clinical_geval.measure(tc1)
        p1_clinical = clinical_geval.score
        
        # Phase 2 clinical
        tc2 = LLMTestCase(input=question, actual_output=phase2_answer, expected_output=expected, retrieval_context=context_list)
        clinical_geval.measure(tc2)
        p2_clinical = clinical_geval.score
        
        # Faithfulness (Phase 2 only)
        faithfulness_metric.measure(tc2)
        faithfulness = faithfulness_metric.score
        
        # Answer Relevancy
        relevancy_metric.measure(tc2)
        relevancy = relevancy_metric.score
        
        # Contextual Recall
        recall_metric.measure(tc2)
        ctx_recall = recall_metric.score
        
        # Store result
        result = {
            'idx': idx,
            'question': question[:200],
            'p1_clinical': p1_clinical,
            'p2_clinical': p2_clinical,
            'rag_lift': p2_clinical - p1_clinical,
            'faithfulness': faithfulness,
            'relevancy': relevancy,
            'ctx_recall': ctx_recall,
            'p1_bleu4': p1_det['bleu_4'],
            'p2_bleu4': p2_det['bleu_4'],
            'p1_f1': p1_det['f1'],
            'p2_f1': p2_det['f1']
        }
        results.append(result)
        
        # Checkpoint every N samples
        if (idx + 1) % CHECKPOINT_EVERY == 0:
            with open(checkpoint_file, 'w') as f:
                json.dump({'results': results}, f)
            print(f"  💾 Checkpoint saved at sample {idx+1}")
            
    except Exception as e:
        print(f"  ❌ Error on sample {idx}: {e}")
        continue

print(f"\n✅ Evaluation complete: {len(results)} samples")

## 7️⃣ Analyze Results

In [ ]:
import pandas as pd
import numpy as np

df = pd.DataFrame(results)
print(f"📊 Results Summary (n={len(df)})")
print("="*60)
print()

# Key metrics
print("CLINICAL CORRECTNESS:")
print(f"  Phase 1 (No RAG):  {df['p1_clinical'].mean():.1%}")
print(f"  Phase 2 (With RAG): {df['p2_clinical'].mean():.1%}")
print(f"  RAG Lift:          +{df['rag_lift'].mean():.1%}")
print()

print("FAITHFULNESS (Phase 2):")
print(f"  Average: {df['faithfulness'].mean():.1%}")
print()

print("CONTEXTUAL RECALL:")
print(f"  Average: {df['ctx_recall'].mean():.1%}")
print()

print("SCORE DISTRIBUTION:")
print(f"  Perfect (100%):  {(df['p2_clinical'] >= 1.0).sum()} ({(df['p2_clinical'] >= 1.0).mean():.0%})")
print(f"  Passing (≥70%):  {(df['p2_clinical'] >= 0.7).sum()} ({(df['p2_clinical'] >= 0.7).mean():.0%})")
print(f"  Failing (<50%):  {(df['p2_clinical'] < 0.5).sum()} ({(df['p2_clinical'] < 0.5).mean():.0%})")

In [ ]:
# Visualize
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Plot 1: Clinical Correctness Distribution
axes[0].hist(df['p1_clinical'], bins=10, alpha=0.5, label='No RAG', color='red')
axes[0].hist(df['p2_clinical'], bins=10, alpha=0.5, label='With RAG', color='green')
axes[0].set_xlabel('Clinical Correctness')
axes[0].set_ylabel('Count')
axes[0].set_title('Score Distribution')
axes[0].legend()

# Plot 2: RAG Lift by Sample
axes[1].bar(range(len(df)), df['rag_lift'], color=['green' if x > 0 else 'red' for x in df['rag_lift']])
axes[1].axhline(y=0, color='black', linestyle='-', linewidth=0.5)
axes[1].set_xlabel('Sample')
axes[1].set_ylabel('RAG Lift')
axes[1].set_title('RAG Improvement by Sample')

# Plot 3: Key Metrics
metrics = ['Clinical\nCorrectness', 'Faithfulness', 'Contextual\nRecall']
values = [df['p2_clinical'].mean(), df['faithfulness'].mean(), df['ctx_recall'].mean()]
colors = ['green' if v >= 0.7 else 'orange' if v >= 0.5 else 'red' for v in values]
axes[2].bar(metrics, values, color=colors)
axes[2].set_ylim(0, 1)
axes[2].set_title('Key Metrics (Phase 2)')
for i, v in enumerate(values):
    axes[2].text(i, v + 0.02, f'{v:.0%}', ha='center', fontweight='bold')

plt.tight_layout()
plt.show()

In [ ]:
# Find worst performing samples
print("❌ WORST PERFORMING SAMPLES (need attention):")
print("="*60)
worst = df.nsmallest(5, 'p2_clinical')
for _, row in worst.iterrows():
    print(f"Sample {row['idx']}: {row['p2_clinical']:.0%} clinical, {row['ctx_recall']:.0%} recall")
    print(f"  Q: {row['question'][:80]}...")
    print()

## 8️⃣ Save Final Results

In [ ]:
# Save final results
final_results = {
    'num_samples': len(results),
    'config': {
        'top_k': TOP_K,
        'llm_model': LLM_MODEL,
        'embed_model': EMBED_MODEL
    },
    'summary': {
        'p1_clinical_mean': df['p1_clinical'].mean(),
        'p2_clinical_mean': df['p2_clinical'].mean(),
        'rag_lift_mean': df['rag_lift'].mean(),
        'faithfulness_mean': df['faithfulness'].mean(),
        'ctx_recall_mean': df['ctx_recall'].mean(),
        'passing_rate': (df['p2_clinical'] >= 0.7).mean(),
        'perfect_rate': (df['p2_clinical'] >= 1.0).mean()
    },
    'results': results
}

with open(OUTPUT_FILE, 'w') as f:
    json.dump(final_results, f, indent=2)

print(f"✅ Results saved to {OUTPUT_FILE}")

# Clean up checkpoint
if checkpoint_file.exists():
    checkpoint_file.unlink()
    print(f"🗑️ Checkpoint file removed")

---
## 📊 Quick Stats

Run this cell anytime to see current progress:

In [ ]:
if results:
    n = len(results)
    p2_scores = [r['p2_clinical'] for r in results]
    faith_scores = [r['faithfulness'] for r in results]
    recall_scores = [r['ctx_recall'] for r in results]
    
    print(f"""📊 CURRENT PROGRESS: {n}/{NUM_SAMPLES} samples ({n/NUM_SAMPLES*100:.0f}%)
    
    Clinical Correctness (With RAG): {sum(p2_scores)/n:.1%}
    Faithfulness:                    {sum(faith_scores)/n:.1%}
    Contextual Recall:               {sum(recall_scores)/n:.1%}
    
    Passing (≥70%): {sum(1 for s in p2_scores if s >= 0.7)}/{n} ({sum(1 for s in p2_scores if s >= 0.7)/n*100:.0f}%)
    Perfect (100%): {sum(1 for s in p2_scores if s >= 1.0)}/{n} ({sum(1 for s in p2_scores if s >= 1.0)/n*100:.0f}%)
    """)
else:
    print("No results yet. Run the evaluation cells above.")